USING ARIMA MODEL TO PREDICT TIME SERIES DATA FORTHE NEXT 7 DAYS

In [1535]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [1536]:
#loading the dataset
df = pd.read_csv('IndianWeatherRepository.csv',index_col='last_updated',parse_dates=True)


In [1537]:
#remove redundant attributes
col_to_remove=['country','latitude','longitude','timezone','last_updated_epoch','temperature_celsius','wind_mph','wind_direction','pressure_in','precip_in','feels_like_celsius','feels_like_fahrenheit','visibility_miles','gust_mph','uv_index','gust_kph','sunrise','sunset','moonrise','moonset','moon_phase','moon_illumination','location_name','air_quality_Carbon_Monoxide', 'air_quality_Ozone','air_quality_Nitrogen_dioxide','air_quality_Sulphur_dioxide','air_quality_PM2.5','air_quality_PM10','air_quality_us-epa-index','air_quality_gb-defra-index']
df = df.drop(columns=col_to_remove)
df

,region,temperature_fahrenheit,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,visibility_km
last_updated,,,,,,,,,,
2023-08-29 10:45:00,Madhya Pradesh,81.5,Partly cloudy,20.5,281,1008.0,0.00,67,26,10.0
2023-08-29 10:45:00,Madhya Pradesh,81.5,Sunny,15.5,287,1008.0,0.00,70,19,10.0
2023-08-29 10:45:00,Madhya Pradesh,79.3,Partly cloudy,18.4,317,1009.0,0.00,70,51,10.0
2023-08-29 10:45:00,Madhya Pradesh,78.1,Cloudy,16.9,297,1009.0,0.00,76,65,10.0
2023-08-29 10:45:00,Madhya Pradesh,81.0,Cloudy,16.2,274,1009.0,0.00,74,82,10.0
...,...,...,...,...,...,...,...,...,...,...
2023-12-19 00:45:00,Uttar Pradesh,55.4,Mist,6.8,250,1020.0,0.00,77,0,2.2
2023-12-19 00:45:00,Mizoram,45.7,Clear,4.0,106,1018.0,0.00,79,4,10.0
2023-12-19 00:45:00,Tamil Nadu,71.9,Partly cloudy,8.6,19,1014.0,0.00,77,47,10.0


In [1538]:
#extract the rows whose region name is Andaman and Nicobar Islands
condition=df['region']=='Andaman and Nicobar Islands'
df = df[condition]
df

,region,temperature_fahrenheit,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,visibility_km
last_updated,,,,,,,,,,
2023-08-29 10:45:00,Andaman and Nicobar Islands,82.0,Light rain shower,36.7,259,1009.0,0.30,80,70,10.0
2023-08-30 08:45:00,Andaman and Nicobar Islands,81.1,Light rain shower,22.0,269,1010.0,1.60,79,75,10.0
2023-08-31 05:15:00,Andaman and Nicobar Islands,82.2,Cloudy,31.7,242,1009.0,0.00,83,69,10.0
2023-09-01 05:15:00,Andaman and Nicobar Islands,81.7,Moderate or heavy rain shower,40.0,243,1010.0,3.90,82,83,7.0
2023-09-02 05:15:00,Andaman and Nicobar Islands,80.6,Moderate or heavy rain shower,33.1,237,1009.0,3.30,83,90,7.0
...,...,...,...,...,...,...,...,...,...,...
2023-12-15 00:30:00,Andaman and Nicobar Islands,82.7,Partly cloudy,21.2,66,1010.0,0.00,78,33,10.0
2023-12-16 01:00:00,Andaman and Nicobar Islands,82.0,Patchy rain possible,22.0,49,1009.0,0.52,80,84,9.0
2023-12-17 00:30:00,Andaman and Nicobar Islands,82.1,Patchy rain possible,25.2,48,1009.0,0.18,82,89,10.0


In [1539]:
#different weather names used
distinct_values = df['condition_text'].unique().tolist()
print(distinct_values)
print("There are "+str(len(distinct_values))+" distinct weather types in the dataset")

['Light rain shower', 'Cloudy', 'Moderate or heavy rain shower', 'Torrential rain shower', 'Thundery outbreaks possible', 'Partly cloudy', 'Patchy rain possible', 'Clear', 'Light drizzle', 'Patchy light drizzle', 'Patchy light rain', 'Heavy rain']
There are 12 distinct weather types in the dataset


In [1540]:
#percentage of missing values
missing_values=df.isnull()
miss_percent=(missing_values.sum()/len(df))*100
missing_data=pd.DataFrame({'missing percent': miss_percent})
print(missing_data)

                        missing percent
region                              0.0
temperature_fahrenheit              0.0
condition_text                      0.0
wind_kph                            0.0
wind_degree                         0.0
pressure_mb                         0.0
precip_mm                           0.0
humidity                            0.0
cloud                               0.0
visibility_km                       0.0


In [1541]:
#identify and remove duplicated rows
duplicated_rows=df.duplicated(keep=False)
print(duplicated_rows)
duplicate_data=df[duplicated_rows]
print(duplicate_data)
df=df.drop_duplicates(keep='first')
print('number of rows: ',df.shape[0])

last_updated
2023-08-29 10:45:00    False
2023-08-30 08:45:00    False
2023-08-31 05:15:00    False
2023-09-01 05:15:00    False
2023-09-02 05:15:00    False
                       ...  
2023-12-15 00:30:00    False
2023-12-16 01:00:00    False
2023-12-17 00:30:00    False
2023-12-18 00:45:00    False
2023-12-19 00:30:00    False
Length: 108, dtype: bool
Empty DataFrame
Columns: [region, temperature_fahrenheit, condition_text, wind_kph, wind_degree, pressure_mb, precip_mm, humidity, cloud, visibility_km]
Index: []
number of rows:  108


In [1542]:
#data type of each row
data_types = df.dtypes
print(data_types)

region                     object
temperature_fahrenheit    float64
condition_text             object
wind_kph                  float64
wind_degree                 int64
pressure_mb               float64
precip_mm                 float64
humidity                    int64
cloud                       int64
visibility_km             float64
dtype: object


In [1543]:
'''my_column = df['last_updated']
modified_column = my_column.apply(lambda x: x[:-5])
df['last_updated'] = modified_column
df
'''

"my_column = df['last_updated']\nmodified_column = my_column.apply(lambda x: x[:-5])\ndf['last_updated'] = modified_column\ndf\n"

In [1544]:
#convert the int64 data to float64
df['humidity'] = df['humidity'].astype('float64')
df['wind_degree'] = df['wind_degree'].astype('float64')
df['cloud'] = df['cloud'].astype('float64')

data_types = df.dtypes
print(data_types)


region                     object
temperature_fahrenheit    float64
condition_text             object
wind_kph                  float64
wind_degree               float64
pressure_mb               float64
precip_mm                 float64
humidity                  float64
cloud                     float64
visibility_km             float64
dtype: object


ENCODING CONDITION TEXT

In [1545]:

from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the column and transform the values
df['condition_text'] = encoder.fit_transform(df['condition_text'])
df['condition_text'] = df['condition_text'].astype(int)

# Access the encoded classes
encoded_classes = encoder.classes_
#print("Encoded Classes:", encoded_classes)

# Decode the encoded values to get the original labels
decoded_labels = encoder.inverse_transform(df['condition_text'])
#print("Decoded Labels:", decoded_labels)

NORMALISE BEFORE CLASSIFICATION

In [1546]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Select only the numerical columns
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the numerical columns and transform the values
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


In [1547]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

# Assuming df is your time series DataFrame with a column named 'cloud'

# Fit the model
stepwise_fit = auto_arima(df[ 'cloud'], trace=True, suppress_warnings=True)

# Forecast the next 7 days
forecast, conf_int = stepwise_fit.predict(n_periods=7, return_conf_int=True)

# Display the forecast
print("Forecast for the next 7 days:")
print(forecast)

# Display the confidence interval
print("Confidence Interval for the next 7 days:")
print(conf_int)


Performing stepwise search to minimize aic


 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=10.892, Time=0.30 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=8.513, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=6.085, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=6.156, Time=0.03 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=215.639, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=8.083, Time=0.05 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=7.963, Time=0.14 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=9.313, Time=0.17 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=55.025, Time=0.01 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.765 seconds
Forecast for the next 7 days:
108    0.649552
109    0.611980
110    0.604447
111    0.602937
112    0.602634
113    0.602573
114    0.602561
dtype: float64
Confidence Interval for the next 7 days:
[[0.17521062 1.12389423]
 [0.12819847 1.09576102]
 [0.12028993 1.08860365]
 [0.11876455 1.08710846]
 [0.11846115 1.08680627]
 [0.11840042 1.08674559]
 [0.

ARIMA FOR NEXT 7 DAYS

In [1548]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Set the date column as the index
#df['last_updated'] = pd.to_datetime(df['last_updated'])
#df.set_index('last_updated', inplace=True)

# Select the parameters for prediction
parameters = ['temperature_fahrenheit', 'wind_kph', 'wind_degree', 'pressure_mb',
              'precip_mm', 'humidity', 'cloud', 'visibility_km']

# Create an empty DataFrame to store predictions
predictions_df = pd.DataFrame()
# Iterate over each parameter
#temperature
train_data=df[:-7]['temperature_fahrenheit']
test_data=df[-7:]['temperature_fahrenheit']
model = ARIMA(train_data, order=(1, 0, 0))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

predictions_df['temperature_fahrenheit'] = predictions


#wind
train_data=df[:-7]['wind_kph']
test_data=df[-7:]['wind_kph']
model = ARIMA(train_data, order=(1, 1, 1))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

predictions_df['wind_kph'] = predictions


#wind deg
train_data=df[:-7][ 'wind_degree']
test_data=df[-7:][ 'wind_degree']
model = ARIMA(train_data, order=(0, 1, 1))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

predictions_df[ 'wind_degree'] = predictions


#pressure
train_data=df[:-7]['pressure_mb']
test_data=df[-7:]['pressure_mb']
model = ARIMA(train_data, order=(1, 0, 0))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

predictions_df['pressure_mb'] = predictions


#preci
train_data=df[:-7]['precip_mm']
test_data=df[-7:]['precip_mm']
model = ARIMA(train_data, order=(1, 0, 0))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

predictions_df['precip_mm'] = predictions


#humidity
train_data=df[:-7]['humidity']
test_data=df[-7:]['humidity']
model = ARIMA(train_data, order=(1, 1, 1))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

predictions_df['humidity'] = predictions


#cloud
train_data=df[:-7]['cloud']
test_data=df[-7:]['cloud']
model = ARIMA(train_data, order=(1 ,0 ,0))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

# Print the predictions
predictions_df['cloud'] = predictions


#visibility 
train_data=df[:-7]['visibility_km']
test_data=df[-7:]['visibility_km']
model = ARIMA(train_data, order=(0, 0, 0))  # Adjust the order as needed
model_fit = model.fit()

# Make predictions for the next 7 days
predictions = model_fit.forecast(steps=7)

# Print the predictions
predictions_df['visibility_km'] = predictions
# Print the predictions DataFrame
print(predictions_df)

# Calculate accuracy metrics
mae = mean_absolute_error(test_data, predictions)
mse = mean_squared_error(test_data, predictions)
rmse = mean_squared_error(test_data, predictions, squared=False)

# Print the accuracy metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

     temperature_fahrenheit  wind_kph  wind_degree  pressure_mb  precip_mm  \
101                0.310936  0.301360     0.154831     0.739119   0.037053   
102                0.303637  0.292177     0.154831     0.769742   0.046232   
103                0.302288  0.287165     0.154831     0.782685   0.048602   
104                0.302038  0.284429     0.154831     0.788156   0.049214   
105                0.301992  0.282935     0.154831     0.790468   0.049372   
106                0.301984  0.282120     0.154831     0.791445   0.049413   
107                0.301982  0.281675     0.154831     0.791858   0.049423   

     humidity     cloud  visibility_km  
101  0.405414  0.631005       0.922025  
102  0.423389  0.610362       0.922025  
103  0.431123  0.605429       0.922025  
104  0.434451  0.604251       0.922025  
105  0.435883  0.603969       0.922025  
106  0.436499  0.603902       0.922025  
107  0.436764  0.603886       0.922025  
Mean Absolute Error (MAE): 0.06913216920089048


CLASSIFICATION PART

In [1549]:
data=df[:-7]
finaltest=df[-7:]

DECISION TREE CLASSIFIER

In [1550]:
# we are now going to approach the solution using DECISION TREE CLASSIFIER
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Load the dataset
X = df.iloc[:, [1,3,4,5,6,7,8,9]].values
y = df.iloc[:, 2].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

# Create a decision tree classifier
clf = tree.DecisionTreeClassifier(random_state=33)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

Accuracy: 90.9090909090909 %


In [1551]:

# we are now going to approach the solution using DECISION TREE CLASSIFIER
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Load the dataset
X = df.iloc[:, [1,3,4,5,6,7,8,9]].values
y = df.iloc[:, 2].values
mi=0
mj=0
max_acc=0       
for i in range(0,100):
    for j in range(0,100):
        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

        # Create a decision tree classifier
        clf = tree.DecisionTreeClassifier(random_state=j)

        # Train the classifier on the training data
        clf.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = clf.predict(X_test)

        # Calculate the accuracy of the classifier
        accuracy = accuracy_score(y_test, y_pred)
        if (accuracy*100)<100 and accuracy > max_acc:   # If this is the best model so far
            max_acc = accuracy     # Save the maximum accuracy
            mi = i                
            mj = j

print("Accuracy:", max_acc*100,"%")
print("Random State Indexes: ",mi, "and", mj)



Accuracy: 95.45454545454545 %
Random State Indexes:  6 and 0


In [1552]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Select only the numerical columns
numerical_columns = predictions_df.select_dtypes(include=['float64', 'int64']).columns

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the numerical columns and transform the values
predictions_df[numerical_columns] = scaler.fit_transform(predictions_df[numerical_columns])
pred=clf.predict(predictions_df)
#print(pred)

# Decode the encoded values to get the original labels
decoded_labels = encoder.inverse_transform(pred)

# Print the decoded labels
print("Decoded Labels:", decoded_labels)

Decoded Labels: ['Cloudy' 'Heavy rain' 'Patchy light drizzle' 'Patchy light drizzle'
 'Patchy light drizzle' 'Patchy light drizzle' 'Patchy light drizzle']


In [1553]:
#last_7_rows = df.tail(7)
#print(last_7_rows['condition_text'])
# Decode the encoded values to get the original labels
#decoded_labels = encoder.inverse_transform(finaltest['condition_text'])

# Print the decoded labels
#print("Decoded Labels:", decoded_labels)
finaltest['condition_text']
finaltest

,region,temperature_fahrenheit,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,visibility_km
last_updated,,,,,,,,,,
2023-12-13 01:00:00,Andaman and Nicobar Islands,0.316239,9,0.290404,0.268786,0.666667,0.094989,0.409091,0.728261,0.875
2023-12-14 00:45:00,Andaman and Nicobar Islands,0.358974,6,0.464646,0.225434,0.888889,0.000000,0.363636,0.304348,1.000
2023-12-15 00:30:00,Andaman and Nicobar Islands,0.410256,6,0.444444,0.179191,0.888889,0.000000,0.363636,0.271739,1.000
2023-12-16 01:00:00,Andaman and Nicobar Islands,0.350427,9,0.464646,0.130058,0.777778,0.038893,0.454545,0.826087,0.875
2023-12-17 00:30:00,Andaman and Nicobar Islands,0.358974,9,0.545455,0.127168,0.777778,0.013463,0.545455,0.880435,1.000
2023-12-18 00:45:00,Andaman and Nicobar Islands,0.376068,6,0.381313,0.156069,0.888889,0.000000,0.454545,0.217391,1.000
2023-12-19 00:30:00,Andaman and Nicobar Islands,0.367521,9,0.262626,0.205202,1.000000,0.021690,0.363636,0.836957,1.000
